In [ ]:
import numpy as np
import os, random
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from PIL import ImageStat,ImageEnhance,Image,ImageDraw,ImageFont
import numpy as np
if not os.path.exists("Art_Dataset/Recog_images"):
                os.makedirs("Art_Dataset/Recog_images")
if not os.path.exists("Art_Dataset/Recog_Gt"):
    os.makedirs("Art_Dataset/Recog_Gt")

In [ ]:
def adjust_brightness(image):
    #Find brightness of image
    #temp = image.convert('L')
    stat = ImageStat.Stat(image)
    brightness = (stat.mean[0]/255)
    BR =[1.70,1.80,1.90,2.0,2.10]
    Brt = np.random.choice(BR,1,replace= False)
    brt = Brt[0]
    enhancer = ImageEnhance.Brightness(image)
    if brightness > 0.10:
        image = enhancer.enhance(brt-brightness)    
    return image

In [ ]:
def image_generate():
    img_number = 1
    #imagefiles = [file for file in os.listdir("../Dataset/resized image/") if file.endswith('.jpg')]
    for i in range(10000):
        #lis = imagefiles[i]
        name = "pillow_"+str(img_number)
        fontsize = [70,80,90,100,120]
        fill_color = ["black"]      
        color = np.random.choice(fill_color,1,replace= False)
        color = color[0]
        image =Image.new('RGBA', (1000, 300), (255,255,255))
        #image = Image.open("../Dataset/resized image/{}".format(lis)).convert("RGB")
        # image shifting 
        a,b = 1,0
        c = [0,1,2,3,4,5,-1,-2,-3,-4,-5] #left/right (i.e. 5/-5)
        LR_select = np.random.choice(c,1,replace = False)
        LR = LR_select[0]
        d,e = 0,1
        f = [0,1,2,3,4,5,-1,-2,-3,-4,-5] #up/down (i.e. 5/-5)
        UD_select = np.random.choice(c,1,replace = False)
        UD = UD_select[0]
        image = image.transform(image.size, Image.AFFINE, (a, b, LR, d, e, UD))
        image = adjust_brightness(image)
        # adding brightness
        
        
        img_width, img_height = image.size   # Get dimensions
        with open('Art_Dataset/Recog_Gt/{}.txt'.format('gt_'+name),'w',encoding='utf-8') as txtfile:
            drawer = ImageDraw.Draw(image)

            flists =[f for f in listdir("OCR-Fonts/") if isfile(join("OCR-Fonts/", f))]
              # dynamic choice of font 
            fontlist = np.random.choice(flists,1,replace = False)
            fsize = np.random.choice(fontsize,1,replace = False)
            image_size = (img_width,img_height)

            left_width, left_height =20,50 
            font = ImageFont.truetype("OCR-Fonts/"+fontlist[0], fsize[0])
            string ="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            string_list = list(string)
            string_first =  np.random.choice(string_list,2,replace = False)
            string_second =  np.random.choice(string_list,2,replace = False)
            string_third =  np.random.choice(string_list,3,replace = False)
            concat_first = ''.join(string_first)
            concat_second = ''.join(string_second)
            concat_third = ''.join(string_third)
            final_string = '{} {} {}'.format(concat_first,concat_second,concat_third)
            width, height = drawer.textsize( final_string, font=font,direction="ltr")

            drawer.text(( left_width,  left_height), final_string, fill=color, font=font,direction="ltr")
            # drawing rectangle surrounding text
            lengthx = left_width + width 
            lengthy = left_height + height
            txtfile.write("{},{},{},{},{},{},{},{},{}\n".format(left_width,left_height,
                                                                lengthx,left_height,
                                                                lengthx,lengthy,
                                                                left_width,lengthy,final_string
                                                               ))
        image.save('Art_Dataset/Recog_images/{}.jpg'.format(name),"PNG")
        img_number+=1

In [ ]:
image_generate()

In [ ]:
##########  crop images and add noise ###################

In [ ]:
import os 
from PIL import Image
import cv2

In [ ]:
import numpy as np
import os
import cv2
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [ ]:
imagelists = [ file for file in os.listdir("Art_Dataset/Recog_images/") if file.endswith('.jpg')]
gtlists = [gt for gt in os.listdir("Art_Dataset/Recog_Gt/") if gt.endswith('.txt')]
print(len(gtlists))
print(gtlists[0])
print(len(imagelists))
if not os.path.exists("Recognition_crop_images"):
                os.makedirs("Recognition_crop_images")
if not os.path.exists("Crop_noise_images_second"):
                os.makedirs("Crop_noise_images_second")


In [ ]:
number = 1
for image in imagelists:
    img = Image.open("Art_Dataset/Recog_images/"+image)
    for gt in gtlists:
        new_gt=gt[3:][:-4]
        #print(new_gt)
        if (new_gt==image[:-4]):
            f= open("Art_Dataset/Recog_Gt/"+gt,'r',encoding="utf-8")
            for line in f.read().splitlines():
                coordinate = line.split(',')
                x1 = int(coordinate[0])
                y1 = int(coordinate[1])
                x2 =int(coordinate[4])
                y2 =int(coordinate[5])
                word = coordinate[-1]
                crop_img = img.crop((x1,y1,x2,y2))
                crop_img.save("Recognition_crop_images/{}_{}.jpg".format(word,number),"PNG")  
                number+=1

In [7]:
import numpy as np
import os
import cv2
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))for i in image.shape]
        out[coords] = 1
        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [8]:
# for rotate image check
import numpy as np
import cv2
import matplotlib.pyplot as plt
imagelists = [ file for file in os.listdir("Art_Dataset/noise_pica_img/") if file.endswith('.png')]
if not os.path.exists("Art_Dataset/pica_noise_img"):
                os.makedirs("Art_Dataset/pica_noise_img")
print(len(imagelists))


5000


In [9]:
for image in imagelists:
    img = cv2.imread("Art_Dataset/noise_pica_img/{}".format(image))
    #blurImg = cv2.blur(img,(3,3))
    noi = ['s&p','gauss','s&p']
    n = np.random.choice(noi,1,replace = False)
    noise_sp = noisy(n,img)
    cv2.imwrite("Art_Dataset/pica_noise_img/{}".format(image),noise_sp)
    

/home/nplab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/nplab/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
################   checking resize and boundary box after resize #####################################

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt
img = Image.open("resize.jpg")
area = (300, 333, 351, 368)
cropped_img = img.crop(area)
cropped_img.save("crop.jpg")
#plt.imshow(cropped_img)

In [ ]:
###########  resize images and resize ground truth ########################################

In [ ]:
import os
gtfiles = [file for file in os.listdir("Art_Dataset/Gt_resize/") if file.startswith('gt_')]
print(len(gtfiles))

In [ ]:
import os
gtfiles = [file for file in os.listdir("Art_Dataset/All_gt/") if file.endswith('.txt')]
if not os.path.exists("Art_Dataset/Gt_resize"):
                os.makedirs("Art_Dataset/Gt_resize")
for gt in gtfiles:
    print(gt)
    with open("Art_Dataset/Gt_resize/{}".format(gt),"w") as f:
        with open("Art_Dataset/All_gt/{}".format(gt),"r") as readgt:
            for line in readgt.readlines():
                cordinate = line.split(',')
                x1 = int(cordinate[0])//2
                x2 = int(cordinate[1])//2
                x3 = int(cordinate[2])//2
                x4 = int(cordinate[3])//2
                x5 = int(cordinate[4])//2
                x6 = int(cordinate[5])//2
                x7 = int(cordinate[6])//2
                x8 = int(cordinate[7])//2
                word = str(cordinate[8])
                f.write("{},{},{},{},{},{},{},{},{}\n".format(x1,x2,x3,x4,x5,x6,x7,x8,word))


In [ ]:
import cv2
import os
import numpy as np
if not os.path.exists("Art_Dataset/Image_resize"):
                os.makedirs("Art_Dataset/Image_resize")
imagelist = [file for file in os.listdir("Art_Dataset/All_img/") if file.endswith('.jpg')]
for image in imagelist:
    img = cv2.imread('Art_Dataset/All_img/{}'.format(image)).astype(np.uint8)
    img_h,img_w,_=img.shape
    iresized_image = cv2.resize(img, (img_w//2, img_h//2))
    cv2.imwrite("Art_Dataset/Image_resize/{}.jpg".format(image[:-4]),iresized_image)

In [ ]:
##########  convert rgb images into 8bit gray scale #####################

In [ ]:
import cv2
import os
import numpy as np

In [ ]:
imagelist = [file for file in os.listdir("Art_Dataset/Com_img/") if file.endswith('.ppm')]
print(len(imagelist))
if not os.path.exists("Art_Dataset/com_gray"):
    os.mkdir("Art_Dataset/com_gray/")

In [ ]:
for image in imagelist:
    img = cv2.imread("Art_Dataset/Com_img/{}".format(image)).astype(np.uint8)
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    new_name = image[:-4]
    cv2.imwrite("Art_Dataset/com_gray/{}.png".format(new_name),img_gray)

In [ ]:
########  vocabulary generation from string #######################################

In [ ]:
strg = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
strlist = list(strg)

In [ ]:
with open('vocab_3.txt','w') as f:
    for i in range(len(strlist)):
        stri1 = strlist[i]
        for j in range(len(strlist)):
            stri2 = strlist[j]
            for k in range(len(strlist)):
                stri3 = strlist[k]
                con_strg = '{}{}{}'.format(stri1,stri2,stri3)
                f.write('{}\n'.format(con_strg))


In [ ]:
with open('vocab_2.txt','w') as f:
        for i in range(len(strlist)):
            stri1 = strlist[i]
            for j in range(len(strlist)):
                stri2 = strlist[j]
                con_strg = '{}{}'.format(stri1,stri2)
                f.write('{}\n'.format(con_strg))


In [ ]:
# generate vocab_2 17576 lines 
count = 1
with open('vocab_2_final.txt','w') as writ:
    while True:
        fr= open('vocab_2.txt','r')
        if count==29792:
            break
        for line in fr.read().splitlines():
            writ.write('{}\n'.format(line))
            count+=1
            if count==29792:
                break


In [ ]:
##########  add number #####################################

In [ ]:
strg1 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
strlist1 = list(strg1)
strg2 = '12345'
strlist2 = list(strg2)

In [ ]:
with open('vocab_num_2_1.txt','w') as f:
        for i in range(len(strlist2)):
            stri1 = strlist2[i]
            for j in range(len(strlist1)):
                stri2 = strlist1[j]
                con_strg = '{}{}'.format(stri2,stri1)
                f.write('{}\n'.format(con_strg))


In [ ]:
with open('vocab_num_3_2.txt','w') as f:
    for i in range(len(strlist2)):
        stri1 = strlist2[i]
        for j in range(len(strlist1)):
            stri2 = strlist1[j]
            for k in range(len(strlist1)):
                stri3 = strlist1[k]
                con_strg = '{}{}{}'.format(stri2,stri3,stri1)
                f.write('{}\n'.format(con_strg))
